In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import os
import time
import unicodedata

In [5]:
main_url="https://townhall.virginia.gov/L/Comments.cfm?GdocForumID=1953"

orig_url="https://townhall.virginia.gov/L/GDocForum.cfm?GDocForumID=452"

In [6]:
html_content = requests.get(main_url).text
soup = BeautifulSoup(html_content, "lxml")
main_tab = soup.find('table')

In [73]:
# save off raw html
with open("raw_html.txt", "w") as text_file:
    text_file.write(html_content)

In [ ]:
with open('raw_html.txt', 'r') as file:
    html_content = file.read()

In [28]:
df = pd.read_html(str(main_tab))[0]

In [57]:
urls = [re.search('.+=([0-9]+)', r.a['href']).group(1) for r in main_tab.find_all('tr')]
df['commentId'] = urls[1:]

In [66]:
df.head(3)

,Comment Title,Commenter,Date,commentId,full_comment
0,I am strongly opposed.,Latosha Misciagna,9/26/22 4:27 pm,136285,
1,Awful legislation!,Vincent J. Albiani,9/26/22 4:27 pm,136284,
2,This is a terrible policy decision.,Gaye Degrazio,9/26/22 4:27 pm,136283,


In [41]:
df.tail(3)

,Comment Title,Commenter,Date
7313,Transgender pronkun,Anonymous (128950),9/26/22 5:24 am
7314,Wholly support the 2022 guidance,"LH, Fairfax County Public Schools mother",9/26/22 2:37 am
7315,Protect Kids,Cassidy Moulton-Hill,9/26/22 12:30 am


In [72]:
# loop over all entries, update in place
df['full_comment'] = ''
for i in range(df.shape[0]):
    url=f"https://townhall.virginia.gov/L/viewcomments.cfm?commentid={df.loc[i,'commentId']}"
    soup2 = BeautifulSoup(requests.get(url).text, "lxml")
    df.loc[i,'full_comment'] = soup2.find(attrs={"class": "divComment"}).text

In [75]:
df.to_csv('1953_comments.csv')

## Alt method - full comments page, paginated

In [7]:
base_url = "https://townhall.virginia.gov/L/ViewComments.cfm?GdocForumID=1953"
params = {"vPage": 1,
  "vPerPage": 999,
  "sub1": "go"}

In [ ]:
pagecount = 999
page = 1

dfc = pd.DataFrame()

while page < pagecount:
    r = requests.post(base_url, data=params)
    try:
        pagecount = int(soup.find(id="page").div.a.text)
    except:
        pagecount = 0
    print(f"page {page}/{pagecount}")
    page += 1
    

In [8]:
r = requests.post(base_url, data=params)

In [53]:
soup = BeautifulSoup(r.text, "lxml")

In [50]:
# find total number of pages
pagecount = int(soup.find(id="page").div.a.text)
pagecount

18

In [52]:
box = soup.find_all(attrs={"class": "Cbox"})[0]
comment = {"Date": unicodedata.normalize("NFKD", box.contents[1].div.get_text(strip=True)),
           "Commenter": box.contents[3].contents[2].strip(),
           "Comment Title": box.contents[3].find_all('strong')[1].get_text(strip=True),
           "Comment": box.find(attrs={"class":"divComment"}).get_text(strip=True),
           "CommentID": int(box.find(attrs={"class":"linklight"}).get_text(strip=True))}
comment

{'Date': '9/26/22  12:30 am',
 'Commenter': 'Cassidy Moulton-Hill',
 'Comment Title': 'Protect Kids',
 'Comment': 'This attempt to keep trans children from affirming their gender in the most vulnerable place they can be is a clear disregard for children’s lives and an attempt at nothing but conveying control over children with an extremely partisan belief.Trans children are more likely to commit suicide over cis children. To declare any type of protection for children and then disregarding their right to be and simply just use a bathroom (a place that should be of extremely little concern of adults) makes the original argument disingenuous at best.Protect children by allowing them the respect to make decisions regarding their own mental health and their decision to choose where to mind their own business without adults obsessing over their genitalia.',
 'CommentID': 128948}

In [43]:
int(box.find(attrs={"class":"linklight"}).text.strip())

'128948'

In [40]:
box

<div class="Cbox" id="cbox128948">
<div style="float: right; text-align: right;">
<div style="background-color: white; border: 1px solid #cccccc; padding: 4px">9/26/22  12:30 am</div>
</div>
<div>
<strong>Commenter:</strong>
          Cassidy Moulton-Hill 
          <br/><br/>
<strong>Protect Kids</strong>
</div>
<div style="clear: right"> </div>
<div class="divComment">
<!DOCTYPE html>

<p>This attempt to keep trans children from affirming their gender in the most vulnerable place they can be is a clear disregard for children’s lives and an attempt at nothing but conveying control over children with an extremely partisan belief. <br/>Trans children are more likely to commit suicide over cis children. To declare any type of protection for children and then disregarding their right to be and simply just use a bathroom (a place that should be of extremely little concern of adults) makes the original argument disingenuous at best. <br/>Protect children by allowing them the respect to make